In [5]:
import tweepy as tw
import pandas as pd
import datetime
import csv
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import networkx



In [6]:
API_KEY = ''
SECRET_KEY = ''
BEARER_TOKEN = ''
ACCESS_TOKEN = ''
SECRET_TOKEN = ''

try:
    auth = tw.OAuthHandler(API_KEY, SECRET_KEY)
    auth.set_access_token(ACCESS_TOKEN, SECRET_TOKEN)
    api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    print("Authentication Successfull")
except:
    print("Error: Authentication Failed")

Authentication Successfull


In [7]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [8]:
today = datetime.date.today()
yesterday= today - datetime.timedelta(days=1)
end = yesterday
start = end - datetime.timedelta(days=1)
print(start, end)

2021-09-08 2021-09-09


In [10]:
search_words = "#asıolmayacağım -filter:retweets"
tweets_list = tw.Cursor(api.search, q=search_words,tweet_mode='extended', lang='tr', since = start, until = end).items(1000)
tweets_no_urls = [remove_url(tweet._json["full_text"]) for tweet in tweets_list]

In [7]:
# Create textblob objects of the tweets
sentiment_objects_tur = [TextBlob(tweet) for tweet in tweets_no_urls]

sentiment_objects_tur[0].polarity, sentiment_objects_tur[0]

(0.0, TextBlob("Second BioNtech shot appointment acquired"))

In [8]:
sentiment_objects = []
try:
    for x in sentiment_objects_tur:
        try:
            if (x.detect_language() != 'en'):
                sentiment_objects.append(x.translate(to="en"))
            else:
                sentiment_objects.append(x)
        except:
            pass
    #sentiment_objects = [x.translate(to="en") for x in sentiment_objects_tur]
except Exception as e:
    print(e)
    pass
sentiment_objects[0].polarity, sentiment_objects[0]

(0.0, TextBlob("Second BioNtech shot appointment acquired"))

In [9]:
# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

sentiment_values[0]

[0.0, 'Second BioNtech shot appointment acquired']

In [10]:
# Create dataframe containing the polarity value and tweet text
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

sentiment_df.head()

,polarity,tweet
0,0.000000,Second BioNtech shot appointment acquired
1,0.157143,areearly3 nichpic TimJDillon The Pfizerbiontec...
2,0.153680,pepeMcGee billbowtell Oh right So Pfizer only ...
3,0.000000,Pfizer BioNTech kicked off the debate over boo...
4,0.175000,Let me also add that Moderna and BioNTech have...


In [11]:
sentiment_df.to_csv("data/textblob_deneme.csv", sep = '|')